### Data Reading

**Read the data from ADFS- Bronze and store in Dataframe**

In [0]:
df_main = (
    spark.read.format("parquet")
    .option("inferSchema", True)
    .load("abfss://bronze@azureproject8940.dfs.core.windows.net/raw")
)

In [0]:
df_main.display()

#### Transformations

** Now take the model ID column- Split the value before the Hyphen and create column and store the value as model category**

In [0]:
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.sql.window import Window

In [0]:
df_main = df_main.withColumn("Model Category", split(col("Model_ID"), "-")[0])
df_main.display()

**Type casting - Unit column convert to string- this is for just testing not include in data frame**

In [0]:
## df_sample=df_main.withColumn('Units_Sold',cast('Units_Sold', StringType())). - Option 1 ##
df_sample =df_main.withColumn('Units_Sold',col('Units_sold').cast(StringType()))
display(df_sample)

#### Arithmetic Operations

**Calculate the Per unit revenue - Reveune/Unit solds**

In [0]:
df_main =df_main.withColumn('Revene_Per_Unit',col('Revenue')/col('Units_Sold'))
df_main.display()

#### Ad hoc


**SC1 - How many unites sold in each branch and yearwise**

In [0]:
df_aggr_test=df_main.groupBy(
  "Year", "BranchName"
).agg(sum(col("Units_Sold")).alias("Total Units Sold")).sort('Year','Total Units Sold', ascending=[1, 0])

display(df_aggr_test)

Databricks visualization. Run in Databricks to view.

#### Data writing in Silver table

In [0]:
df_main.write.format("parquet").mode("overwrite").option(
    "path", "abfss://silver@azureproject8940.dfs.core.windows.net/carsales"
).save()

#### Quering Silver data

In [0]:
%sql
select * from parquet.`abfss://silver@azureproject8940.dfs.core.windows.net/carsales`